In [1]:
!git clone https://github.com/openai/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [2]:
import os
os.chdir("gpt-2")

In [3]:
!pip3 install tensorflow-gpu==1.14.0

     |████████████████████████████████| 377.0MB 46kB/s 
     |████████████████████████████████| 491kB 43.8MB/s 
     |████████████████████████████████| 3.2MB 53.4MB/s 
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2


In [5]:
!pip show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wheel, absl-py, six, termcolor, tensorboard, keras-applications, google-pasta, gast, tensorflow-estimator, numpy, protobuf, astor, wrapt, keras-preprocessing, grpcio
Required-by: fancyimpute


In [6]:
!pip3 install -r requirements.txt

In [7]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 1.31Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 61.1Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.39Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:16, 84.7Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 10.2Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 57.9Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 61.4Mit/s]                                                       


In [8]:
!export PYTHONIOENCODING=UTF-8

In [9]:
os.chdir('src')

In [10]:
import json
import os

In [11]:
import numpy as np

In [12]:
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [12]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 42kB/s 
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [13]:
import model, sample, encoder

In [14]:
from tensorboard import version
print(version.VERSION)

1.14.0


In [15]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [ ]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)

Example Text Created.. 
input : Once a upon a time

In [ ]:
"""One a upon a time in the 1400s, artisans who cleaned beach sand cleaned a remedy for ailing feet. For centuries, women tucked 
their feet so they didn't rub sand into the joints and served their feet with herbs and poultices from the soil. 
The practice stuck, and today women are known as floor-cleaning women.

Now, just about every coast is regarded as a head-dive destination. In Bay Diomede, a Red Sea resort, fish 
dumpers wait for their cue to dip after a performing or sightseeing tour. Jenny Anderson stopped just outside 
the historic answer, one of 10 such bile dumpers along this island: a collection of cavelike mobile fish lagoons 
where residents have built fried chicken stands, mirrors set up to peer in pond and stock the barges regulated by 
sailboats. The boomers guilty of spilling their feathers talk loudly, endlessly to Curtis and friends when Dr. Madison 
Ness comes along and announces the element of tiprofloxacin to lead to dolphin poo treatments. They shake their arms in 
empty offense over the illuminations in Nature Or None (Music Sister/Dovesaver DVD: 200 out of 750 feedback reviews) as 
scientific controversy rages over whether coffee is the active ingredient in the alcoholic barista's quantum biscuit flavored 
beverage concoction (610 out of 724 reviews).

"I haven't heard the term 'elite' said it twice," Miss Miller says of Ms. Miller,"""